### Environment Preparation 

* prepare session and bucket 
* prepare necessary parameters 

In [12]:
import sagemaker 
import boto3 
sess = sagemaker.Session()
BUCKET = sess.default_bucket()

In [21]:
BUCKET

'sagemaker-us-west-2-334326423663'

In [13]:
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]


In [1]:
!ls 


a2i-object-detection-and-retraining.ipynb  process_a2i_results
augmented.manifest			   README.md
invoke_image_object_detection		   sample-a2i-images
prepare-lambda-functions.ipynb


### Setup lambda functions and SQS 
We use a lambda function ```invoke_image_object_detection``` to 
1. trigger the sagemaker endpoint to do inference
2. save the image to s3 and trigger a2i process 
3. send a2i and s3 information to SQS 

And, we use a lambda function ```process_a2i_results``` to 
1. listening to SQS and process a2i results 
2. trigger retraining pipelines 

The following steps creates lambda functions 

* copying codes to s3 
* create lambda functions by codes on s3 
* setup environment variables 


In [10]:
%%bash -s "$BUCKET" 
cd invoke_image_object_detection 
zip -r invoke_image_object_detection.zip  .
aws s3 cp invoke_image_object_detection.zip s3://$1/lambda/
cd ../process_a2i_results
zip -r process_a2i_results.zip  .
aws s3 cp process_a2i_results.zip s3://$1/lambda/

updating: lambda_function.py (deflated 55%)
updating: pass_tasks2sqs.py (deflated 43%)
upload: ./invoke_image_object_detection.zip to s3://sagemaker-us-west-2-334326423663/lambda/invoke_image_object_detection.zip
  adding: prepare_data.py (deflated 67%)
  adding: model.py (deflated 45%)
  adding: lambda_function.py (deflated 62%)
upload: ./process_a2i_results.zip to s3://sagemaker-us-west-2-334326423663/lambda/process_a2i_results.zip


In [18]:
!aws lambda create-function --function-name invoke_image_object_detection --zip-file fileb:///home/ec2-user/SageMaker/mlops_incremental_learning/invoke_image_object_detection/invoke_image_object_detection.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role arn:aws:iam::$account_id:role/TeamRole


{
    "FunctionName": "invoke_image_object_detection",
    "FunctionArn": "arn:aws:lambda:us-west-2:334326423663:function:invoke_image_object_detection",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::334326423663:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 1813,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-02T03:07:26.175+0000",
    "CodeSha256": "lKGsuC9x0t53yRtvIwR9CHRNpLfUOHGvqZM6xBxBNQ4=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "bf8c941d-0775-4d5e-beab-791f6224dcf4",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


#### Configure lambda function - invoke_image_object_detection 
* you can also do it by command line - 
```
aws lambda update-function-configuration --function-name my-function \
    --environment "Variables={BUCKET=my-bucket,KEY=file.txt}"
```    

![configure environment variable](./content_image/setup_env_vars_for_lambda2.png)


In [19]:
!aws lambda create-function --function-name process_a2i_results --zip-file fileb:///home/ec2-user/SageMaker/mlops_incremental_learning/process_a2i_results/process_a2i_results.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role arn:aws:iam::$account_id:role/TeamRole

{
    "FunctionName": "process_a2i_results",
    "FunctionArn": "arn:aws:lambda:us-west-2:334326423663:function:process_a2i_results",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::334326423663:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 2528,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-02T03:07:42.224+0000",
    "CodeSha256": "ETBIo6C7GoQaVH40i/ZL+XZGyFVV7kF71ct/wH2H7n8=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "0844706f-6052-4c24-b219-f98748cd90f0",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


#### Configure lambda function - process_a2i_results
![configure environment variable](./content_image/setup_env_vars_for_lambda3.png)

### Set up SQS 

* we use SQS to pass the information between lambda functions in this solution 
* set up SQS and attach it as a trigger to a lambda function 



In [20]:
sqs = boto3.resource('sqs')
queue = sqs.create_queue(QueueName='a2itasks', Attributes={'DelaySeconds': '0'})
print(queue.url)



https://us-west-2.queue.amazonaws.com/334326423663/a2itasks


In [23]:
queue

sqs.Queue(url='https://us-west-2.queue.amazonaws.com/334326423663/a2itasks')

In [25]:
!aws lambda create-event-source-mapping --function-name process_a2i_results  --batch-size 10 \
--event-source-arn arn:aws:sqs:us-west-2:$account_id:a2itasks



An error occurred (ResourceConflictException) when calling the CreateEventSourceMapping operation: An event source mapping with SQS arn (" arn:aws:sqs:us-west-2:334326423663:a2itasks ") and function (" process_a2i_results ") already exists. Please update or delete the existing mapping with UUID 946bfd25-9461-4a51-876d-339e7d108211


### Test Material 
```
{
  "content": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRUZGRgYGBgYGBoYGBoYGBgYGhgZGRgYGBgcIS4lHB4rIRgYJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISHjQrJCQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAMMBAgMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAEAAIDBQYBB//EADkQAAEDAgQEBAMGBgIDAAAAAAEAAhEDIQQSMUEFBlFhEyJxgTKhsUKRwdHh8BQVUmJygiPxB5Ky/8QAGQEAAwEBAQAAAAAAAAAAAAAAAAECAwQF/8QAJBEAAgICAwACAgMBAAAAAAAAAAECESExAxJBE1EiMgRhgXH/2gAMAwEAAhEDEQA/AABjTG6aKyAZWC67EhcDbDsWLn2QtWoVC3EAqUEKbYdgd9KdUFX4c0o6u6NEmGVomIqTw0dENW4SDstA4BDuTUmBS0uDiVLU4SNCrunAXKjgm5sADDcHaRGUKj4hwlzXWG61rKkBQYhwKcZtOxUYd+FcNk0UndFr3YVpSfgGwtPlY8mNLT0SAK1zeHN6JzeGN6I+VBZksvUKMha+rwodFEzgwnRC5UKyhZmy2BQtQkm62/8ALQGqlxXB5d5QlHkVgmZ9JXL+COCHfwp4WqnFjtFckjv5a9NdgHBO0FoDSUpw7ui4aThsU7QyNJSCiei4WEbItAMBSKSUIASSRCSAEkkkgDXGuVzxVASmBy5EhBza6kOKIQTXJr3lNxFQd/FSnnGQFVhyeRZLqJhYxqe3EKtIU9JKgDhXSa8ygy6EvGRQFk+quEygGVC4wFe4HglR0F8U2/3fEfRv5otIqKlJ0kQ4PDh13TlHTUnoFd/wdF7LMLSdHAkkHvJgqQiixuUbD7h17pYbiDG7SD1tH5LGUm3g7YcUVGmsgNPgDyfLUZHW/wAwmYzg1anfLnb/AFMkx6iJC0+EOe9o7z+KPZiabCJqNHUG6PkfoPgi9HnLnwuU6q9QxWDoV2Fr8pkGHNABaToQUPgeWMMwAtYXnWXmflp8k1NUYvgdmCbRe4SGOI6wYXKZG4Xrjn5GEaNANsrdIVZR5awrpc5mYuuczjN+gBgeyblHwn4JHmtVzUK9jVveJcq4UuysL6ZO4OZs9CHfmlheWcOxozAVXaOLiW/+rZshTig+CVmDZRaVFWwjdoXoLKGHpkinTDHdYLj9529E19U7NbHVoAjuRCPmXhov4rrLPPsNwd9Qwxhd6Cw9ToEY/k6pEuLG9nOP1AIWsdiHTrPr+Sc7ENcMpspfO/DSP8WK2zB47l+pREvZDTo5pDmH/YWB7FVVTBjovTWPyTo5hEOY64c3cELMce4c2k/yXY8B7JuQD9knqD+C0hy9jDm4emVoyD+HjonNwQ6K2cJ2SFNbdznKl3D5GiFOAutHkEIOsxNTHZWfwASRmUrqfYLDGUJTH0YKJpPSqGVnYwRzEwMRLmKNzYRYEMQkSpMspgYiwIi1PYYTnNUYclQrHuN04XSaJTiIQInw9QsJe0eYAlvr+cSh8XzBVcfjOnX8FzOg8XQnzN9x36hHVXk24pUqHjiT+pU9HiD5EEkuP00CqQYN0XhjLgRtum4pG/Z/ZccR5kfTbkYb6SCbnc+nRZ3+ZVnuBL3EylisI74j6W2RXC8KHvAPwi59AqqEY3RPaUpbwekcv8ScWMzAxb1J6wt3hXktDoi2i8xw/GKVDLmgWnsBt7q84TzjTqmA6DMRtA3XG00rrB0YeLNgAXlzSYGVxI9GkoWhXd11+SJ5ZqNqB5I1zN9oVFUxDWPex5gA2J0hS1UVL7sE/wAnEvThcw819x++qz/N1Ooyk57ZMAzHyPqoqfMtNj8hqNE6EuhaV9dlRmUwQ6x/RKsZQ7aPH8BzkWuAcTGkm/4LT8O4uysYY4B/TZ3p+SxfNPAhTqPDLQ4/mFV8IbUaZBjKZB6EdCuh8UJR7QZHyyUqkj1l7JvEOF+nuFVcRxTWAuecsW/6R2D4wx1NheQX5fNpqLGR+9VVcb4hScIDG37yfumAsEldGzdK7HYfEB1Jry17swJADg2GCwJsTJIMe3VVfF6pc5l5bkGWdQNCD3kIetxBxblFhA+URHawQufSSt4xrJx83KpKkyVRl9106KGbrQ5iYlNc0LhNlHmQJnMgSTl1FsQxlRPc9KrRhOpM7JlCYDqlUKMY2yGfR1QDQM1yTinChdJ1IpEkVRQEoh7CozQKYDfEXDUXXUCuHDlNIdHJXXFMa0p5TBCYAdQnuaFxgXXBSxj2MEQpOG4bK5xm0W97/gowYRVB3lJlTLRpwvNFdj8F4jrHzDrpCs+W+WanjDzNiL7g9h30+9Op4HP5i8NHpeB6LX8Ir0KNMPfNogadSJ7mFPeTj1R0qMU+zN5ytg8gaAIDRfue6B5q4CXguYGk31EiFQYPnOS4spuIsBDgIJMCVpeX+YxiC6i9jvEa0lwdAcWyBItfULX4bhTM/lqdo8pfyjUc5xcXQTldo2zbi3QbEL0Hlrh7WMaA5xDRAzOLr+6t+NcGNsroB+1Egjo4dUM0CiweabWgRJ9FzzlJvq/DaPWrj6ed89tjEubPlLWu++fnqsy58aLU84DNUncMbJtrLjH3ELLuYtOJLqc/M25UJlSVMAhskKRj1ZiSOXIXQ5LMihCkppUkrhRQUNldyroCdMIoBmQpKXOEk6YqQhVlPY8KLw4TgEgoKZXCe9wIQDmFRlzkUCDMi5lUdOoVIXooZwNvdEeGIQbqhldNYwl1AkewJll2kZ1XKohOgGeCCoX0EQEg2UACikniknudCZnQA0005php/f73XAV10EaxEEe179kpLBUHUkF4WuAWg2uB6g6fijuNYx9TDupC0GZGttB81VcWwrm0i9mkNe0ja6bw/iQqAtLocR5mnQ9x1CcI1+RvOVqjPYZxaS3PAMAwT5TJE+0fNek/+I8C5uIdWNUuY1mT+0ucQXBpJkgR2uvOcVSptcZdfNcC8X6jZej8kcwUKNFzc7Z11yxGgJN102YHp3NXFGUcK+pJMRA1MyNFh38S8anTqmxJyR/cQTMeyp+LcXdxF4oUSW0ZzPfoXwW/1XiJ7kxoNdlyvwSS177MYZYztoCfYBcvNBykkjfjkoxyUXH+DEMNR5Azhoa37XwgG+5Xn+Jplji06g/9Fem88uL8Y0D4W0wBqbkmbfcvPeOUyHj/AB1PqUsRfVCmrj2YCxdKa0JwKujnOTCRK48pjE6AJY+y4EwBdDUBY8PSe6UxwhNzpCsV0kvEC4jsItxQldGHurQUhoFK3DBTZp1KR7OygqM7K6xVMdFBTpgo7B1KtrCphRI1CsG0bpzxKVh1K4YeV11EKwDAntoAosVFO+idkvDO6tBSE3ThRBTsKKhrF1jJVwcM0KIUgEdvA6lX4EqB1FXooKF2HCaYqKplJONIaQrNmGB0RLMFNgJhDd4KSoi4dRz0sr7QS2+mUjT0us3U4Pkqmm6A5zXZO9iT8gfvWqxOJyQ0jaD0lA8QoGs1rmOAqMByE76QAdiri+rLu2ZkcEOkbj63Wm4TwSmKbxiDkDv+Ng3L3tcWH08s+yhwPNLaYIq4QmoCIc13lJ7tIstNw59TFllSrTbSptcHBhIzPI0uR0kbfEffZtGlxStGo5Q5dpYZjAYLy3M+e9hbQDULR1MSxj8jdSJPtsqHhwqOqOc9rhmFyLgCIG+yu6GFp2MS7QTss1oybtlPx3AzUa/KYy3cBN9gs5zByz4tPMweZoJZ/deXMPrt3XqXgiIOioeKUgHQ0RfQad1hzRcfyTNYSUl1Z4ezCHQjRNfhey2vNHDPDf4jR5HmT0Dzcj3uVngJKcZWrMJR6uipOGURoFW9SneEhTA1VWTRU+GUvDKtHsErhYITsKKt7ZUIZdW3gjRdZhQCkKiq/h+xSV/4DUkBQeydVwYqLJ+HqWQ1dt1kW2OqvlDiWlTNlJ7eqdBY9olNewhcBI0Uk9U6GRN1RTBZQ09ZTnPG6SQqGVmrlJymzghQYcyYVD0Odqk9kXUlamBdNAzeil0sieBuddYQpXUQdCh3UyClGSehJ4Jm0byp8HZ+utvyQZqGLIjhzC57fWVcbspBGNwuYEbqh8CPKTBBsfpC1WIYZsEFiKDHxnEEHUReFpJDRDgwx13tBdu4tABP7hX/AA54eHUnR8JyWgAi7dFFgeDtPwmw7D9J2ROH4U5rpDmxmsTMiDe4uNxupbaKikyy4RiX5AxpfrveOy1OEpwJce/Qqnw4ywA6ZvbtrfZWmHcSeoVRFIsWGUPxDCBzZA8wuETSTqxsfRXOKlFpkxbTwYzjODD6b2ET5SW/5C4XnIIb6r07HOgx2K874jhQDmA66dVxccqbRvyxtdgCq64JTKhulVqhOosButbOehGFG6l0UtWn0TaQKOyYtkGUyngKQtukXABJyoUmkR5ikn5uySnuT2D3EBMIBUlOnm12UlKnJ00Qk0WlggZTJcOinLBOiKFMLjqZBkXQUC1RA0QzKbiVYam6Ip5WppCor6dM9FFVwpVg/wCKdihnvgmUk2NkDKDhpom5YMoh2IMABRYhx2CdhSHsEi6ieSNkwVSLFEZ5Gil2weUDsc4qSgSTB0RTQMkkKNjxeE44EkkQVG3MKfhNQh8dj9FD4l0qZh2b1Vp07GlbLx7utwi8Ng21D8J/1k/NC8NcH6wJ6q+/k5DbSJ6GxXRtD0GcNwQZ5XAjYZomehg2Vo2iwGIF/wB7rP4Z+TyukHrNj6zoiGVMzhc/eoWBsKxNZjHEQQNo2JROG4nSFhmn/FR4/DtIzbIdlNpNvdVoRo6FYOEt0TsTUAbqqrDPyugaKTHPdIt+ScpVFhGNsqOKN8xOyyHEsoYT0d9bfitjxIyDbQLG8bHkcB2+oleev2OmX6FFVc03hQvrCLJswLqNlOdFvSOR5HGuUn4qEQWNAAUL6QzRsmkhJEuGqA6rkCUjQACYwQjAMfASTfDHVdRQYLV1AsGZS4Zh1K7XJMAqdlOQob8G/oY8rrRNgoatM6dE6gYN0k3YqZ12G1umuGVhUue5hQvqzbYaqrKqkCUMzutlPVph0HfdE0y0adLqFo887FIWyM0w1wJ0TKwzOFrKxq0xv7KAggTCd+DS8IG4e8kWClZTgF0WRAe0sJNioXPLmgDRCCiBtbMCIsmsokbJ9NwY2SLSp6r9C3dF+j3kHdoLaJuNgttspGVQTA21UmJALHAawUWEcMfy/wCV2ZzrDYR8zqvQsFjmPYBoR7ELFcvYIvYSW7fhCt8Dw17I809Jm/a6h8soukdKgpLJd43CAtzWWcw9fLULdpWjq4kMovzkCAd1g34uX5mrpcrSZjVOjWcW4jDMo1gFZ6hxZwdrH5LtWqXAON7KnxVTKZCbb2SjUt4kYDswgant6rmH44+q8AAwPU/UarJsxudjmt+KQ33NgfqtfwSm1rAGiwGvU3ue65+abqkb8UVst8U2Wz1aZWO4405JA+IgBbl7paFhOZJDwxn2Jn3/AEWaWQm/xaM/icNbulQow2US9zgbiU1rXAXFlon9nKo0DvbFkOHkOhWzKQ33TBgW5rXVJjoHLbXTcRTltkY6gu0sOfZFhTsqfAPVJW2XskixUi0xNNoEoZmINxsEXVF9fZcr4fSNwpoojw+YDM4WOiY+mbuUhaWsh5k7BDGsbNE7J2BPQpF11ABE23hWAEXE2GnVNa+ZJHeEMAdrCRA1KjewsIEI6k9okxB2Ujhm9OqT+gKr+Jc4y4aaIwMloKDreVxbsiWYkmGgbITBsixFIWaTGYp4pgGBpomPZL2z9ko6vVAdLR2TDwEdQtBvdReDY9Roj2sIAdsUI6pBch6Ahw9K56zdTGBMCbGfuTsKYc4kahPADWGR5nAj71KQKrL7l+uMhbpb6K8wjg45Tvp6rz7g+LLX/rZbHD1SRLetuxSkns6lnBTc+4ktc2ju5oe70kgfQrLMqWhaXn6jm8LEaOA8CoDsfM9h/wDv5LIStLXhjK/S8p4mGBVON8036rj6ktDeikwGFNR7GAwXECTt1Psm5dsErBY8h8vGs573khgIA7ka+116GOFeHGUS3r27qy4Nw9lGk2mwQAPck3JPeSjy20dlq+FNZ2NcrWjNYp+VpHRYvGed7ndSTKv+N4rzvpg6Egn20+aochgCNd1yqLtj5HdIFpYa8qDEsdtBurxtIBgEKsbTJcRBsrZD0QmkIEld8rW21KJGHBaUIyh5ydgEqFsTaTnRGv4JPYS4ibiy7hn5Sb3UrGfaPcosLAPDd1SRXjDskpsLRZOy5p2At6phY4X13soalBpEOdMaAdUTTpwASU78E8AeOY98ZbRclJjIZLrkHXsj2vF56dIlQCsxxiPKQhO0CyDOxT5GQW6rj6paQALm8o52Roa0aaJ2IcA3L133SavYmrIW1IcCRqNFI6mbAm0yY2GyFrNI8wM2hcw1Qxc/EIgpppDqjmOZNgNxB6qYPDQDG111gmGkjsigWFhYRfcpoa/sDxFAkAt3Ejsm0BaHbI+ocryGXsIH1KHqSLRrc9ymxNHK1VobF5GnSUPSbml3TVKrTJiGz1SwRnMGjVTkYmOBGvmCcxs63Kc+kGuEtk9dkZSoZrsItr3TX0CZS1aJY8dDcHv0K9J5W4eGMOaC4w70svO+MVXCm5wjyOafuN16by2ZoMeT8QmT02W/Ck7sJSdIF504U2rhK8Nl7WF7Y1LmDMBbUkAj3Xj+AJeMoBJGkX/7XvzKrSYDgT6rEcxcuspVRXpeRr3f8jRGUO/qaNpOo6p8kE8oSk9Hn7qBGv6q85RwZfiaZIsCXH2B/RS4pmYg6iTeNI0Wo5CwZyvqOGrsrf8AEakep+ijjjchtmybog+L4o06T3jZpj1hGtCoObHODGR8JfleOxEj5hdL0QYfEvIaHmS57jJ9bk/RPfWOUEjQW/fsiuN0w0sDfhAJM9SqmpUdodtLLhSawzWT7Owo4kua0gqKnWIJFpO/Zcp0BldJhxiFG6mdBroTO6b+xaHY5xbGUz1UNFtySNpKTKLgMsiZTcNVcHunTf0CS/slHXsBEgKei5rmEdFHVuCR8M2UT3Whvb37JWCwxuXskjG0XwPKkn1GOZTJIaPsiSVM5pI8okk23v0TGvJGZrSLkO7jpKcxjpgyA3zAQpik0Rs5iGuiLE7t6dVXV6TgczB0kKw8W0kOBOi7lJhgudZ7I65GmBeLmcGvERcdlNXJJ0kDdS0cK0ODyPNJEnS3ULrKbs+YeYGTA09AihtWyZlItYHGJ1ynYIFkuu34h5vWeiMqsc5jnEXFo0tuhKbHAtY0AWtPT1T14JANZ7/EzQQALd1aNcSJLdQCnNfmIkD23jYItktGVwEuNwNh0lLDHvYJQxOd0htxA/RTPpPnQEjYnRDswuXOWzBNvVcwdUvBaTBbr39U442LQTWeAC82AOUDqYuoWGAMgOZ2oAn9hTupjKM1xJ9j1hLh77Py7yJjZV6CYBXD3ENIOt/RdoPcxttydFY4t4BY0C9iSO43QviDxcjT8IMnp1RVDA6+Fzse12pY6B0sYPrKIxvNQo0WU2vAaKTSASPhDToNzYe0qUsJzECQ5sT0jqsVx7hDzVaARGRzBmmMjiTY9QSR6ALTjlVjLvB/+SqjmEljWFrmhzgZsS6C1ojTyg3KvsJzd/EANcWkOny3u4CWuA2vIhYXhnJpMB7iWnUC3uDC1OD4cykcrGRlEAm5deZLlU31Vh2VFg+sfOyLOEgdFt+UCf4ZhdYmY/xBIb8gF589rg62oMQfxWw5e4kxwYxpBjM0xMBwixP+rrdkcLy7E8mtBVNzU5ow7i7QFpHrmH6qehiTlcbE+YtA94H4LPczcWY+gaYcDUztGQXJElaSlgHGjMYnGF5G4M6bKTxRkDRBMx3ChqUpbPwlsTFtUxtGCHToR7rktpheAhxizm6iL7nZVbKD2u1Nzp0V3iCHDPN7QNRG6GZiQ55tobH6pyBjXUjAgibn23lDeH10Fz3CmL5e63l0/ROd5hMRt6KW1QXaAKeKaXuYJLdk3MWlvc2PvuiDRIJbF5F46p9TDicua4+UpJWGy3ZiLC40C6qvKz+o/ckrt/Y8ljiWwbW1+pQ+GeSDJJnXvdJJJaJZeY6g3yCBAbooOL0gDTIES38l1JOWmUAvYIIjdR4Wzvl8kkllHz/RLQ2qLn0H1T8WwZGmL6T/AKlJJOQIFw7bjtP0CN3aerSuJKVofgqf2RsSbKPFMANhrKSSa2J6HYNsi9/N9FC+zXRbyn6pJKvCfoJpsGVhi5YPqVVlgFV9tXD6JJKXtif7FjR+A93ge3RRcwYZnggxcPsekriSqO0UN4V8KO+27s0JJLfm/Vf9BbG1qLYeYvDbyVk34t9LHUKdNxYx1RxLW2Bza/U/eupJQ2UjR4jidVlSnlqESwk6X/5ag/Afcp6dIZ3GLnKSdz5Ekkp/qUypYJeJ3cQe6Kx9qrgNMuiSSyWjN6Ez4PcKFjfID/e76lJJNAywawZXW3UlKmC15jqfkkkmg8BKHxj0/BQYqg3MDH2ep6pJJS0xouaOGblFth16JJJKCj//2Q=="
}
```


### Test Data 
```
{
  "Records": [
    {
      "messageAttributes": {
        "s3-image-path": {
          "stringValue": "s3://sagemaker-us-west-2-230755935769/a2i-results/sample-a2i-images/pexels-photo-276517.jpeg"
        },
        "a2i-task-id": {
          "stringValue": "a78d671e-217f-474b-bc12-c029097fb7bf"
        }
      }
    }
  ]
}
```



## Configure API Gateway 
* Finally, we need an API to have the service accessible
* This API accepts image POST to it and pass the image to ```invoke_image_object_detection```

```curl -X POST -H 'content-type: image/jpeg' --data-binary @$f $OD_API | jq .``` 

* we can create it by console also by aws cli 



#### Select RESTful API 


![configure environment variable](./content_image/build_api_gateway.png)



### Create Resource and Methods 

![configure environment variable](./content_image/create_resource.png)
<br>
![configure environment variable](./content_image/create_method.png)
<br>
![configure environment variable](./content_image/create_method2.png)
<br>



### Configurations for passing the binary content to backend
* Add binary media type in ```Settings``` of this API 
* Configure which attribute to extract and fill it in event in Lambda integration 

![configure environment variable](./content_image/setting.png)
![configure environment variable](./content_image/integration.png)
![configure environment variable](./content_image/setting1.png)

In [5]:
!cat api.sh 

#!/bin/bash

#OD_API='https://2lvd1jdy8l.execute-api.us-west-2.amazonaws.com/test/classify'
OD_API='$1'

for f in $(ls images/*.jpg)
do
    curl -X POST -H 'content-type: image/jpeg' --data-binary @$f $OD_API | jq .
done


In [10]:
!./api.sh "https://2lvd1jdy8l.execute-api.us-west-2.amazonaws.com/test/classify" 

https://2lvd1jdy8l.execute-api.us-west-2.amazonaws.com/test/classify
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61502  100 24901  100 36601   6759   9935  0:00:03  0:00:03 --:--:-- 16694
"{\"prediction\": [[7.0, 0.894026517868042, 0.17456209659576416, 0.0, 0.7324751019477844, 0.9828765392303467], [11.0, 0.09282755106687546, 0.01725822687149048, 0.0, 0.8312187194824219, 0.9899668097496033], [7.0, 0.03428087756037712, 0.19231387972831726, 0.07277275621891022, 0.7797596454620361, 0.5433324575424194], [14.0, 0.029871676117181778, 0.025632724165916443, 0.6540557742118835, 0.27294784784317017, 0.9936500191688538], [7.0, 0.024339014664292336, 0.30609601736068726, 0.09689390659332275, 0.6467463970184326, 0.7659716606140137], [7.0, 0.023789698258042336, 0.3011087477207184, 0.39898017048835754, 0.6980705261230469, 0.9022207260131836], [7.0, 0.023602239787578583, 0.09527771174907

100  221k  100 18271  100  203k  19604   218k --:--:-- --:--:-- --:--:--  237k
"{\"prediction\": [[11.0, 0.816798746585846, 0.2493259608745575, 0.1033935546875, 0.8767216205596924, 1.0], [11.0, 0.05425266921520233, 0.2630082368850708, 0.13556961715221405, 0.6975359916687012, 0.6287321448326111], [7.0, 0.04241780936717987, 0.3321705758571625, 0.18649864196777344, 0.9927867650985718, 0.94593346118927], [14.0, 0.02744041569530964, 0.5629246830940247, 0.23883749544620514, 0.6429172158241272, 0.3346860110759735], [14.0, 0.024824267253279686, 0.36490604281425476, 0.24276630580425262, 0.44247230887413025, 0.33473166823387146], [14.0, 0.02442081645131111, 0.46714091300964355, 0.15621747076511383, 0.5385342836380005, 0.22885383665561676], [14.0, 0.0229374747723341, 0.5611574053764343, 0.2851843535900116, 0.6425700783729553, 0.388445109128952], [11.0, 0.022905860096216202, 0.3403330445289612, 0.13487182557582855, 0.6924458146095276, 0.3981826901435852], [14.0, 0.02280142717063427, 0.515416979789

100 25168  100 18684  100  6484  24779   8599 --:--:-- --:--:-- --:--:-- 33335
"{\"prediction\": [[11.0, 0.7077956795692444, 0.3545849323272705, 0.02328088879585266, 0.8515524864196777, 0.9739731550216675], [14.0, 0.12708190083503723, 0.27156829833984375, 0.10076692700386047, 0.8835328817367554, 0.9305438995361328], [11.0, 0.028891319409012794, 0.47211822867393494, 0.06103205680847168, 0.8775771856307983, 0.588003396987915], [7.0, 0.028096666559576988, 0.45089900493621826, 0.02919355034828186, 0.9760229587554932, 0.9113539457321167], [14.0, 0.02233651466667652, 0.624454915523529, 0.22060677409172058, 0.6872817873954773, 0.3618243634700775], [11.0, 0.02160496823489666, 0.5171141028404236, 0.2423422932624817, 0.8101269602775574, 1.0], [14.0, 0.02069084346294403, 0.6159639358520508, 0.2872444987297058, 0.6873892545700073, 0.391163170337677], [14.0, 0.018511755391955376, 0.6815446019172668, 0.22602948546409607, 0.7374245524406433, 0.3625745475292206], [14.0, 0.018037183210253716, 0.6611202